In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
with open("./data/causal_inference.txt", 'r') as f:
    text = f.readlines()

In [3]:
res = []
for t in text:
    res.append(list(map(int, t.strip('\n').split(" ")[:-1]))) # 最後空文字が入っている。
#     print(t.strip('\n').split(" ")[:-1])

# コロンビア大学で用いられているサンプルデータセット
http://www.cs.columbia.edu/~jebara/6998/hw2.pdf

- the 1st column is the chosen arm
- the 2nd column is binary reward 
- the remaining columns are the context features.

In [4]:
ds = pd.DataFrame(res)
ds.columns = ["arm", "reward"] + [f"feat_{i}" for i in range(100)]

In [5]:
X = ds.loc[:, "feat_0":].values
y = ds.loc[:, "reward"].values
arms = ds.loc[:,"arm"].values

In [6]:
import os, sys

sys.path.append(".")

In [7]:
from src.Arms import ContextualArms

In [8]:
import itertools, os
ds.groupby("arm").mean()["reward"].idxmax()

2

In [9]:
Arms = ContextualArms(n_action=10, max_n_sim=100, X=X, y=y,arms=arms, n_features=X.shape[1])

In [10]:
Arms

In [11]:
d = 100
sigma=1; sigma_0=1
A_inv =sigma_0/sigma * np.eye(d); A_inv
b_t = np.zeros((d, 1))
alpha = 0.05
n_arms = 10
n_sim = 1000
history = dict(chosen_arms=[],rewards=[])
mu_hat = np.zeros((n_arms, 1))

In [12]:
#  Thompson sampling アルゴリズム
for t in range(1, n_sim):
    # 3. draw_theta
    theta = np.random.multivariate_normal(A_inv@b_t.flatten(), sigma**2*A_inv, size=1)
    # 4. A[t]はa_itの転置ver
    # 各アームの特徴量を引く
    # shape : (10, 100) 10 arms, (1, 100)の特徴量ベクトル
    # 1次元目にアーム１から10までの特徴量を順番に格納している。
    A_ = Arms.draw()
    for i in range(n_arms):
        mu_hat[i] = np.dot(A_[i].ravel(), theta.ravel())
    i_star = mu_hat.argmax(0)[0]
    history["chosen_arms"].append(i_star)

    #5. スコア最大の行動 i*　を選択して報酬を観測する。
    reward = Arms.get_reward(index=i_star)
    history["rewards"].append(reward)
    #6.  A-inverse/ bの更新をする。
    a_is = A_[i_star, np.newaxis]
    second_term = (A_inv@a_is.T@a_is@A_inv) / (1+a_is@A_inv@a_is.T)
    A_inv = A_inv - second_term
    b_t += a_is.T*reward    

In [13]:
A_[1, np.newaxis]

array([[ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0,
         3,  5,  0,  0,  7,  7,  0,  0,  0,  0,  0, 23, 31,  0,  3,  4,
         3,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 15, 39,  0,
         0,  0,  9,  2,  1,  3,  0,  6,  0,  0,  0,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  0,  0,  1,  0,  0,  3,  7,  0,  0,  0,  0, 64,
         5,  0,  0,  0,  0,  0,  6,  0,  3,  0,  1,  0,  0,  0, 13,  0,
        24,  2,  0,  0]])

# Logit Regression

- logit モデルは対数事後確率の勾配ベクトルから陽に（解析的に）$\theta$を求めることが難しい。

$- \log\pi (\theta| \{i(s), X(s)\}^t_{s=1})$
$\displaystyle = \frac{\theta^T \theta}{2\sigma^2_0} + \sum^T_{s=1}log(1+\exp(\theta^Ta_{i(s), s})) - \sum^T_{s:X(s)=1}\theta^Ta_{i(s), s} + Constants. $


$G_t(\theta) = - \nabla\log\pi (\theta| \{i(s), X(s)\}^t_{s=1})$
$\displaystyle = \frac{\theta}{2\sigma^2_0} + \sum^t_{s=1}\frac{e^{\theta^Ta_{i(s), s}}a_{i(s), s}}{(1+\exp(\theta^Ta_{i(s), s}))} - \sum^T_{s:X(s)=1}a_{i(s), s}$

- $G_t(\theta) = 0が\theta$について解けない


- そこで、数値計算をする。ニュートン法を用いて、負の対数事後確率を$\hat{\theta}_{MAP}$の周りで２次近似すると、($G_t(\hat{\theta_{MAP}}) = 0$より)

- 負の対数事後確率が多変量正規分布の負の対数ゆうどに近似される。これをLaplas近似というらしい。
    - $\theta_{MAP} \sim N(\hat{\theta_{MAP}}, H_t(\hat{\theta_{MAP}})^{-1})$

In [14]:
# def component_logit_hessian(theta, arm_feature):
#     return np.exp(np.dot(theta.ravel(), arm_feature.ravel()))* arm_feature.T@arm_feature /(1+np.exp(np.dot(theta.ravel(), arm_feature.ravel())))**2
def Hessian_inv(theta, Arms=None):
    assert Arms is not None
    print(theta.shape)
    n_his = len(Arms.history_idx)
    first_term = np.eye(d)/sigma_0
    # self.cum_matrix
    # initial: self.cum_matrix = np.zeros((d, d))
    exp_theta_arm_feats = np.exp(X[Arms.history_idx]@theta.T)
    print(exp_theta_arm_feats.shape)
    sum_past_second_term = exp_theta_arm_feats.reshape((n_his, 1,1)) * (X[Arms.history_idx, :, np.newaxis]@X[Arms.history_idx, np.newaxis, :])
    second_term = np.sum(sum_past_second_term, axis=0)# (100, 100)
    hessian = first_term + second_term

    return np.linalg.inv(hessian)

def Gradient(theta, sigma_0,  Arms):
    '''gradients vector for logit based posterior probability.'''
    first_term = theta/sigma_0
    exp_theta_arm_feats = np.exp(X[Arms.history_idx]@theta.T)
    second_term = np.sum(exp_theta_arm_feats*X[Arms.history_idx]/(1+exp_theta_arm_feats), axis=0, keepdims=True) #1.
    third_term = np.sum(X[Arms.history_idx][y[Arms.history_idx]==1], axis=0, keepdims=True).astype(np.float64)
    # output shape (100, 1)
    return (first_term + second_term - third_term).T


def newton_iteration(theta, Arms, n_iteration=1000, thr=1e-10):
    for _ in range(n_iteration):
        H_inv = Hessian_inv(theta, Arms)
        print(f"iter_{_}:", theta.shape)
        theta_next = theta - (H_inv@Gradient(theta, sigma_0,  Arms)).ravel()
        if np.sum((theta_next - theta)**2) < thr:
            break
        theta = theta_next
        print(f"iter_{_}:", theta.shape)
    return theta_next, H_inv



In [ ]:
d = 100
sigma=1; sigma_0=1
A_inv =sigma_0/sigma * np.eye(d); A_inv
b_t = np.zeros((d, 1))
alpha = 0.05
n_arms = 10
n_sim = 1000
history = dict(chosen_arms=[],rewards=[])
mu_hat = np.zeros((n_arms, 1))

In [15]:
# ロジスティックモデル上のトンプソン抽出
# WRITE CODE
# Newton method
theta, H_inv = newton_iteration(theta, Arms, n_iteration=1000, thr=1e-10)
# 6. draw_theta
theta_tilde = np.random.multivariate_normal(theta.ravel(), H_inv, size=1)
# 各アームの特徴量を引く
# shape : (10, 1, 100) 10 arms, (1, 100)の特徴量ベクトル
# 1次元目にアーム１から10までの特徴量を順番に格納している。
A_ = Arms.draw()
for i in range(n_arms):
    mu_hat[i] = np.dot(A_[i].ravel(), theta_tilde.ravel())
i_star = mu_hat.argmax(0)[0]
history["chosen_arms"].append(i_star)



(1, 100)
(999, 1)
iter_0: (1, 100)
iter_0: (1, 100)
(1, 100)
(999, 1)
iter_1: (1, 100)
iter_1: (1, 100)
(1, 100)
(999, 1)
iter_2: (1, 100)
iter_2: (1, 100)
(1, 100)
(999, 1)
iter_3: (1, 100)
iter_3: (1, 100)
(1, 100)
(999, 1)
iter_4: (1, 100)
iter_4: (1, 100)
(1, 100)
(999, 1)
iter_5: (1, 100)
iter_5: (1, 100)
(1, 100)
(999, 1)
iter_6: (1, 100)
iter_6: (1, 100)
(1, 100)
(999, 1)
iter_7: (1, 100)
iter_7: (1, 100)
(1, 100)
(999, 1)
iter_8: (1, 100)
iter_8: (1, 100)
(1, 100)
(999, 1)
iter_9: (1, 100)
iter_9: (1, 100)
(1, 100)
(999, 1)
iter_10: (1, 100)
iter_10: (1, 100)
(1, 100)
(999, 1)
iter_11: (1, 100)
iter_11: (1, 100)
(1, 100)
(999, 1)
iter_12: (1, 100)
iter_12: (1, 100)
(1, 100)
(999, 1)
iter_13: (1, 100)
iter_13: (1, 100)
(1, 100)
(999, 1)
iter_14: (1, 100)
iter_14: (1, 100)
(1, 100)
(999, 1)
iter_15: (1, 100)
iter_15: (1, 100)
(1, 100)
(999, 1)
iter_16: (1, 100)
iter_16: (1, 100)
(1, 100)
(999, 1)
iter_17: (1, 100)
iter_17: (1, 100)
(1, 100)
(999, 1)
iter_18: (1, 100)
iter_18: (1,

AttributeError: 'dict' object has no attribute 'append'

We can implemented one step learning.
Then 

## If n_sim is sufficiently small, then the draws of theta_tilde wont work because H_inv is not solved.

In [41]:
theta_tilde = np.random.multivariate_normal(theta.ravel(), H_inv, size=1)

LinAlgError: SVD did not converge

In [42]:
H_inv

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

---

In [62]:
# \exp(\theta^T a_{i(s),s})
exp_theta_arm_feats = np.exp(X[Arms.history_idx]@theta.T)
second_term = np.sum(exp_theta_arm_feats*X[Arms.history_idx]/(1+exp_theta_arm_feats), axis=0)

In [83]:
exp_theta_arm_feats

array([[4.22606446e-16],
       [2.26175669e-01],
       [6.66285331e+00],
       [1.34520764e+00],
       [5.72468099e-01],
       [1.31072705e+00],
       [2.54577232e-01],
       [1.29484757e+00],
       [8.84702170e-02],
       [2.28285222e+03]])

In [88]:
X[Arms.history_idx, np.newaxis,:][0]

array([[66,  0,  0, 14,  0,  0,  0,  0,  0, 23,  0,  7, 19,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,
         0,  0,  7,  0,  4,  9,  0,  0,  0,  0,  0, 11,  4,  7,  0, 10,
         0,  0,  0,  0,  0,  0,  0, 38,  0,  1, 12,  8,  0,  0,  0,  0,
         0,  5,  0,  0,  2,  0,  0,  0, 14,  0,  0,  0,  0,  0,  3, 10,
         2,  0,  0,  0,  1,  0,  0,  2,  0,  0,  0,  8,  0,  0,  1,  3,
         6,  0,  0,  0]])

### Hessianの要素を計算する

In [137]:
# np.sum(X[Arms.history_idx, :, np.newaxis]@X[Arms.history_idx, np.newaxis, :], axis=0)
hoge = exp_theta_arm_feats.reshape(10, 1,1) * (X[Arms.history_idx, :, np.newaxis]@X[Arms.history_idx, np.newaxis, :])
np.sum(hoge, axis=0)

In [138]:
(X[Arms.history_idx, :, np.newaxis]@X[Arms.history_idx, np.newaxis, :]).shape

(10, 100, 100)

In [152]:
# 掛け算の動作確認
# 各試行（n_sim）ごとに、(10, 1) × (1, 10)の掛け算がうまくできているか確認
np.all(X[Arms.history_idx[0], np.newaxis].T@X[Arms.history_idx[0], np.newaxis] == (X[Arms.history_idx, :, np.newaxis]@X[Arms.history_idx, np.newaxis, :])[0])


True

In [139]:
# 掛け算の動作確認
# 各試行のnp.exp(X[Arms.history_idx]@theta.T)と上の行列がそれぞれ掛け合わせられてるか確認
np.all(hoge[0] == exp_theta_arm_feats[0, 0] * (X[Arms.history_idx, :, np.newaxis]@X[Arms.history_idx, np.newaxis, :])[0])

True

In [155]:
np.sum(hoge, axis=0)

(100, 100)

In [140]:
(X[Arms.history_idx, :, np.newaxis]@X[Arms.history_idx, np.newaxis, :]).shape

(10, 100, 100)

In [136]:
hoge.shape

(10, 100, 100)

In [110]:
np.all((X[Arms.history_idx, :, np.newaxis]@X[Arms.history_idx, np.newaxis, :])[0] == X[Arms.history_idx[0], :, np.newaxis]@X[Arms.history_idx[0], np.newaxis, :])

True

In [107]:
X[Arms.history_idx[0], :, np.newaxis]@X[Arms.history_idx[0], np.newaxis, :]
# first_term = theta/sigma_0
# exp_theta_arm_feats = np.exp(X[Arms.history_idx]@theta.T)
# second_term = np.sum(exp_theta_arm_feats*X[Arms.history_idx]/(1+exp_theta_arm_feats), axis=0, keepdims=True) #1.
# third_term = np.sum(X[Arms.history_idx][y[Arms.history_idx]==1], axis=0, keepdims=True).astype(np.float64)
# first_term + second_term - third_term

array([[4356,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0]])

In [48]:
(exp_theta_arm_feats*X[Arms.history_idx])[2] == exp_theta_arm_feats[2] * X[Arms.history_idx][2]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [52]:
np.sum(exp_theta_arm_feats*X[Arms.history_idx], axis=0)

array([7.00050632e+03, 6.55363525e+00, 7.78987402e+04, 1.28421873e+02,
       2.28344424e+04, 0.00000000e+00, 2.49038139e+01, 1.33257066e+01,
       0.00000000e+00, 1.15041526e+04, 3.88149911e+04, 2.22655876e+00,
       5.88031060e+01, 1.40247794e+02, 6.66285331e+00, 3.42495686e+04,
       5.09154464e-01, 1.65345183e+01, 1.18737590e+02, 0.00000000e+00,
       8.46030976e+04, 1.10153461e+01, 0.00000000e+00, 2.05610280e+01,
       0.00000000e+00, 8.84406513e+01, 2.33146851e+03, 1.77921582e+01,
       1.57403770e+01, 3.42470215e+04, 2.28310680e+03, 5.09154464e-01,
       1.27288616e+00, 5.30821302e-01, 7.83649383e+00, 3.19645345e+04,
       2.84978677e+01, 2.29830161e+03, 2.65410651e-01, 2.56022022e+03,
       1.36999059e+04, 0.00000000e+00, 2.51997092e+01, 2.20390979e+01,
       2.15688777e+01, 2.17146132e+02, 5.55484765e+01, 2.14604955e+01,
       4.62589630e+03, 4.57027806e+03, 1.46277277e+01, 2.72598713e+01,
       3.13060856e+00, 2.15160233e+01, 1.99885599e+01, 1.03793556e+02,
      

In [57]:
# exp_theta_arm_feats[2]
(exp_theta_arm_feats*X[Arms.history_idx]/(1+exp_theta_arm_feats))[0] == (exp_theta_arm_feats*X[Arms.history_idx])[0]/(1+exp_theta_arm_feats)[0]


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

array([80.07902537,  2.83617888, 76.1638408 , 58.03467343, 13.3484506 ,
        0.        , 10.77747973,  1.73900062,  0.        , 48.95679219,
       22.26810465,  1.49800864, 27.85694738, 60.69422797,  0.86950031,
       20.52711729,  0.40583748,  8.03292197, 17.2349045 ,  0.        ,
       62.39529603,  5.51765069,  0.        ,  2.97255797,  0.        ,
       22.35307394, 22.44831799,  8.26565787,  9.69090105, 18.51368676,
        1.20248088,  0.40583748,  1.0145937 ,  0.48767646,  3.58974398,
       17.23281046, 12.99101484,  8.65064007,  0.24383823, 39.839149  ,
        8.32961103,  0.        ,  4.87145489, 11.19684101,  5.85367997,
       32.85367713, 17.77229189,  9.99881743, 10.00908323,  4.07761416,
        6.10405152,  4.00603761,  1.54030903,  3.82601337,  2.60850092,
       47.35483004,  0.        , 10.86085101, 10.73329428,  9.31451931,
       15.57669426, 10.60062894,  2.11065301,  2.05977525, 44.04225116,
        6.60588943, 14.69022928,  0.36405705,  8.25390759,  8.41

# Calculate Regrets.

In [ ]:
# 関数にするかも
class RegretCalculator():
    '"EnvとAgentのヒストリーを踏まえて、期待リグレットを計算する。"'
    pass

In [37]:
theta.shape

(1, 100)